In [1]:
!pip install selenium

In [2]:
!python --version

import selenium
print(selenium.__version__)

Python 3.13.2
4.29.0


In [3]:
!pip install pandas
!pip install webdriver-manager

In [5]:
!pip install mysql-connector-python

In [6]:
# 웹크롤링 브랜드별 자동차 판매순위 2024년 월별
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import time

# 1. Selenium WebDriver 실행
service = Service(ChromeDriverManager().install())
wd = webdriver.Chrome(service=service)

data_list = []

# 🚗 브랜드별 ID 및 순위 매핑
brand_dict = {
    1: {"name": "현대", "id": 303},
    2: {"name": "기아", "id": 307},
    3: {"name": "제네시스", "id": 304},
    4: {"name": "BMW", "id": 362},
    5: {"name": "벤츠", "id": 349},
    6: {"name": "KGM", "id": 326},
    7: {"name": "르노코리아", "id": 321},
    8: {"name": "테슬라", "id": 611},
    9: {"name": "쉐보레", "id": 312},
    10: {"name": "볼보", "id": 459}
}

# 2. 브랜드별 2024년 데이터 크롤링
for brand_rank, brand_info in brand_dict.items():  # ✅ `rank` → `brand_rank`
    brand_name = brand_info["name"]
    brand_id = brand_info["id"]

    for i in range(2024, 2025):  # 2024년만 크롤링
        for j in range(1, 13):  # 1월부터 12월까지 크롤링

            # 3. 웹페이지 이동
            url = f'https://auto.danawa.com/auto/?Work=record&Tab=Model&Brand={brand_id}&Month={i}-{j:02}-00&MonthTo='
            wd.get(url)

            # 4. 웹페이지 로딩 대기
            try:
                element = WebDriverWait(wd, 10).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, '#autodanawa_gridC > div.gridMain > article > main > div > table.recordTable.model > tbody'))
                )
            except:
                print(f"❌ {brand_name}({brand_id}) {i}-{j:02} 데이터 로드 실패")
                continue  # 다음 월로 넘어감

            # 5. 데이터 크롤링
            rows = element.find_elements(By.TAG_NAME, 'tr')
            sales_month = f'{i}-{j:02}'
            print(f"📅 {sales_month} {brand_name}({brand_id}) 데이터 크롤링 중...")

            for row in rows:
                try:
                    # 다른 class 속성이 있는 경우 건너뜀
                    class_attribute = row.get_attribute('class')
                    if class_attribute and ('sub' in class_attribute or 'model_' in class_attribute):
                        continue  

                    # ✅ 모델 순위 (rank → model_ranking으로 변경)
                    model_ranking = row.find_element(By.CLASS_NAME, 'rank').text.strip()

                    # 모델명
                    model_element = row.find_element(By.CSS_SELECTOR, 'td.title a')
                    model_name = model_element.text.strip()  

                    # 판매량
                    sales = row.find_element(By.CLASS_NAME, 'num').text.split(' ')[0].replace(',', '')

                    # 점유율
                    market_share = row.find_element(By.CLASS_NAME, 'rate').text.replace('%', '')

                    # 전월 판매량 & 증감률 처리
                    last_month_sales, last_month_change = "0", "0"
                    try:
                        last_month_values = row.find_elements(By.CLASS_NAME, 'right')[1].text.split('\n')
                        last_month_sales = last_month_values[0].replace(',', '')
                        last_month_change = ('-' + last_month_values[1].split('▼')[0].strip()) if '▼' in last_month_values[1] else last_month_values[1].split('▲')[0].strip()
                        last_month_change = last_month_change.replace(',', '')
                    except:
                        pass  # 데이터가 없으면 기본값 유지

                    # 6. 데이터 저장 (변경된 변수명 반영)
                    data_dict = {
                        '브랜드 순위': brand_rank,  # ✅ rank → brand_rank
                        '브랜드': brand_name,
                        '브랜드 ID': brand_id,
                        '모델 순위': model_ranking,  # ✅ rank_model → model_ranking
                        '모델명': model_name,
                        '판매량': sales,
                        '점유율': market_share,
                        '전월 판매량': last_month_sales,
                        '전월 대비 증감': last_month_change,
                        '판매월': sales_month
                    }
                    print(data_dict)  # 데이터 확인
                    data_list.append(data_dict)

                except Exception as e:
                    print(f"⚠️ 오류 발생: {e}")
                    continue

# 7. WebDriver 종료
wd.quit()


📅 2024-01 현대(303) 데이터 크롤링 중...
{'브랜드 순위': 1, '브랜드': '현대', '브랜드 ID': 303, '모델 순위': '1', '모델명': '싼타페', '판매량': '8014', '점유율': '20.7', '전월 판매량': '7627', '전월 대비 증감': '387', '판매월': '2024-01'}
{'브랜드 순위': 1, '브랜드': '현대', '브랜드 ID': 303, '모델 순위': '2', '모델명': '투싼', '판매량': '5152', '점유율': '13.3', '전월 판매량': '3588', '전월 대비 증감': '1564', '판매월': '2024-01'}
{'브랜드 순위': 1, '브랜드': '현대', '브랜드 ID': 303, '모델 순위': '3', '모델명': '포터2', '판매량': '4923', '점유율': '12.7', '전월 판매량': '5658', '전월 대비 증감': '-735', '판매월': '2024-01'}
{'브랜드 순위': 1, '브랜드': '현대', '브랜드 ID': 303, '모델 순위': '4', '모델명': '아반떼', '판매량': '4392', '점유율': '11.3', '전월 판매량': '4990', '전월 대비 증감': '-598', '판매월': '2024-01'}
{'브랜드 순위': 1, '브랜드': '현대', '브랜드 ID': 303, '모델 순위': '5', '모델명': '그랜저', '판매량': '3635', '점유율': '9.4', '전월 판매량': '8410', '전월 대비 증감': '-4775', '판매월': '2024-01'}
{'브랜드 순위': 1, '브랜드': '현대', '브랜드 ID': 303, '모델 순위': '6', '모델명': '캐스퍼', '판매량': '3006', '점유율': '7.8', '전월 판매량': '4021', '전월 대비 증감': '-1015', '판매월': '2024-01'}
{'브랜드 순위': 1, '브랜드': '현대', '브랜드 ID'

In [7]:
# MYSQL에 데이터베이스 생성
# CREATE DATABASE IF NOT EXISTS car_sales;
# USE car_sales;

import mysql.connector

# 1️⃣ MySQL 연결 설정
db = mysql.connector.connect(
    host="localhost",
    user="root",      
    password="root1234",   
    database="car_sales"  # 사용할 데이터베이스
)

cursor = db.cursor()

# 2️⃣ 테이블 생성 (한 번만 실행)
cursor.execute("""
    CREATE TABLE IF NOT EXISTS car_sales (
        id INT AUTO_INCREMENT PRIMARY KEY,
        brand_rank INT,  
        brand_name VARCHAR(50),  
        brand_id INT, 
        model_ranking INT,
        model_name VARCHAR(255),
        sales INT,
        market_share FLOAT,
        last_month_sales INT,
        last_month_change INT,
        sales_month VARCHAR(10)
    )
""")

# 3️⃣ 데이터를 MySQL에 저장하는 SQL문
sql = """
    INSERT INTO car_sales 
    (brand_rank, brand_name, brand_id, model_ranking, model_name, sales, market_share, last_month_sales, last_month_change, sales_month)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

# 4️⃣ 리스트 데이터 변환 (`executemany()`로 빠르게 처리)
values = [
    (
        int(data['브랜드 순위']),  
        data['브랜드'],        
        int(data['브랜드 ID']), 
        int(data['모델 순위']),  
        data['모델명'],        
        int(data['판매량']),    
        float(data['점유율']),  
        int(data['전월 판매량']),
        int(data['전월 대비 증감']),
        data['판매월']          
    ) for data in data_list
]

cursor.executemany(sql, values)  # ✅ `executemany()` 사용해서 한 번에 삽입

# 5️⃣ 변경 사항 저장 & MySQL 연결 닫기
db.commit()
cursor.close()
db.close()

print("✅ 데이터 저장 완료! MySQL에서 확인하세요.")


✅ 데이터 저장 완료! MySQL에서 확인하세요.
